# Entrainer les agents addi

## Settings

In [1]:
from google.colab import drive
drive.mount('/content/drive')
%reset -f
%cd /content/drive/MyDrive/RECHERCHE3
%ls
#%cd /content/drive/MyDrive/ml_flux_Burgers2/

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/.shortcut-targets-by-id/17bT9qzfPBT-758cXfyfE3KbFtOk0U2aT/RECHERCHE3
ddqn_lib/  GCN/          ml_flux_euler/  Pinns/            translations/
Euler/     GCN.zip       model_1.h5      popup_lib/
Euler2/    Hamilton/     model_2.h5      transformer/
Euler3/    Hamilton.zip  model_vNet.h5   transformer_old/


In [ ]:
%cat Euler3/param.py

In [3]:
import Euler3.core_solver as core
import Euler3.agent_addi as aga
import Euler3.neural_networks as nn

from Euler3.backend import K
from Euler3.param import Param,Projecter
from Euler3.initial_conditions import *

#from Euler.backend import K
import matplotlib.pyplot as plt
import popup_lib.popup as pop
import numpy as np
import copy
import time

import tensorflow as tf

%reload_ext autoreload
%autoreload 2

Popup loaded!


In [4]:
# Attention: `BC_periodic` rajoute une discontinuité au bord pour chaque fonction non périodique.
flux_corrections = ["flux", "viscosity", "slopes"]
param = Param(nx=1000,nx_ratio=10,
              BC_solver=Param.BC_neumann,
              BC_model=Param.BC_neumann, 
              flux_correction=flux_corrections[2],
              )
              # BC_solver=Param.BC_periodic,
              # BC_model=Param.BC_periodic)
nb_t=1000
k_tf32=K("tf",32)

TypeError: ignored

In [ ]:
default_min_rho = 0.05
default_min_P = 0.05

min_rho = default_min_rho * 10
min_P = default_min_P * 10

batch_size = 100

# pour ne pas dépasser la RAM : nb_t=1000, batch_size = 140, nx=1000 fonctionne

def data_for_train():
    return tf.concat((
        init_periodic(param,4*batch_size//10),
        init_random_sod(param,4*batch_size//10,minimum_E=min_P,minimum_rho=min_rho),
        init_single_contact(param,2*batch_size//10,minimum_P=min_P,minimum_rho=min_rho)
      ), axis=0)

# Il est tout à fait possible de mettre dans val des K-tests déterministes

def data_for_val():
    return data_for_train()

# génération des solutions

def generate_fine_solutions(for_train):
    if for_train:
        w_init=data_for_train()
    else:
        w_init=data_for_val()
    return core.compute_solutions(param, nb_t, w_init,False, k_tf32)

### Définition des familles d'agent

In [ ]:
fam_size=6
#un durée pour que les agents puissent tous faire leur 'nb_optimization' optimizations
#attention, cela dépend beaucoup de la watch_duration
# period_duration="6 steps"#"15 seconds" 
period_duration="12 steps"#"15 seconds" 
all_agents=[]

def family_full(window_size,color):
        name="full_"+str(window_size)
        agents=[]
        # model_struct = (32,64,32)
        model_struct = (32,64,64,32,16)
        for _ in range(fam_size):
            # model=nn.Difference_model_several_convo(param,4)
            # model=nn.Difference_model_tricky(param,window_size)
            model=nn.Difference_model_tricky_no_diff(param,window_size)
                 
            agent= aga.Agent_addi(param, 
                            model,
                            watch_duration=20, #20
                            lossCoef_stab=100, #10.
                            lossCoef_ridge=1e-3,
                            lossCoef_disHLL=1,
                            lossCoef_consistency=1,
                            nb_optimization=15,
                            )
            agents.append(agent)
            all_agents.append(agent)
            
        return pop.Family_trainer(
            agents=agents,
            nb_bestweights_averaged=3,
            nb_strong=3,
            nb_weak=2,
            period_duration=period_duration,
            name=name,
            color=color)

family_trainers=[
                #  family_full(3,"red"),
                 family_full(5,"red"),
                #  family_full(7,"blue"),
                 ]

def load_data():
    ti=time.time()
    W_train = generate_fine_solutions(True)
    W_val = generate_fine_solutions(False)
    print(f"|génération des données, durée: {time.time()-ti:.2f} ",end="")
    
    ti=time.time()
    for agent in all_agents:
        agent.load_WY_train(W_train)
        agent.load_WY_valid(W_val)
    print(f"|load données,durée: {time.time()-ti:.2f}",end="")

### LA BOUCLE

In [ ]:
try:
    for i in range(50):
        load_data()
        for family_trainer in family_trainers:
            family_trainer.period()
except KeyboardInterrupt:
    for family_trainer in family_trainers:
        #pour pouvoir reprendre si on veut
        family_trainer.interupt_period()

|génération des données, durée: 1.29 |load données,durée: 2.17
full_5,échauffement a↗1.1842

### Historique des `famparams`

In [ ]:
def plot_history_famparams():
    # keys=["lossCoef_stab","lossCoef_ridge","lossCoef_disHLL","lossCoef_Laplacian","watch_duration","score"]
    keys=["lossCoef_stab","lossCoef_ridge","lossCoef_disHLL","watch_duration","score"]

    for k in keys:
        fig,ax=plt.subplots()
        for family_trainer in family_trainers:
            family_trainer.plot_metric(k,ax)

    plt.show()
plot_history_famparams()

### Historique des `loss`

In [ ]:
def plot_history_losses():
    # keys=["loss_stab","loss_disHLL","loss_ridge","loss_Laplacian"]
    keys=["loss_stab","loss_disHLL","loss_ridge",
          "loss_stab*coef","loss_disHLL*coef","loss_ridge*coef",
          "score_l1","score_l2","score_linfty"
          ]

    for k in keys:
        fig,ax=plt.subplots()
        for family_trainer in family_trainers:
            family_trainer.plot_metric(k,ax)

    plt.show()
plot_history_losses()

## Choix d'une des familles

### Valeurs des famparams

In [ ]:
family_trainer=family_trainers[0]

for name,agent_w in family_trainer.agents.items():
    print(name)
    print(agent_w.agent.famparams)
    print(agent_w.agent.agent_score())


best_agent=family_trainer.get_best_agent()
# best_agent=family_trainer.agents['a'].agent
print("best agent")
print(best_agent.famparams)
# print(best_agent.agent_score())


## Testons le long du temps

In [ ]:
def compare_along_the_time(W_val, apply_addi_after_nth_time_step=10):
    nb_t = len(W_val)
    
    best_agent.load_WY_valid(W_val)
    res_fine_proj,res_HLL,res_model=best_agent.predict(apply_addi_after_nth_time_step)

    # print(np.sum(W_val[0,0,5:-5,0])/990)
    # print(np.sum(W_val[50,0,5:-5,0])/990)
    # print(np.sum(res_fine_proj[0,0,5:-5,0])/90)
    # print(np.sum(res_fine_proj[50,0,5:-5,0])/90)
    # print(np.sum(res_HLL[0,0,5:-5,0])/90)
    # print(np.sum(res_HLL[50,0,5:-5,0])/90)
    # print(np.sum(res_model[0,0,5:-5,0])/90)
    # print(np.sum(res_model[50,0,5:-5,0])/90)
        
    nb_batch_plot=5
    if nb_batch_plot>W_val.shape[1]:
        nb_batch_plot=W_val.shape[1]
    
    nb_t_plot=3
    #pour avoir le permier temps et le dernier
    t_to_plot=np.floor(np.linspace(0,nb_t-1,nb_t_plot)).astype(int)
    if nb_batch_plot==1:ax=ax[:,tf.newaxis]
    
    fig, ax = plt.subplots(nb_t_plot,nb_batch_plot,figsize=(15,nb_t_plot*2))
    for i,t in enumerate(t_to_plot):
        for j in range(nb_batch_plot):
            ax[i,j].plot(res_fine_proj[t,j,:,0],color="k",label="fine")
            ax[i,j].plot(res_HLL[t,j,:,0],color="b",label="HLL")
            ax[i,j].plot(res_model[t,j,:,0],color="r",label="model")
    ax[0,0].legend()    
    ax[0,0].set_title("Rho")
    fig.tight_layout()    
    
    errors_HLL=tf.reduce_mean(tf.abs(res_fine_proj-res_HLL),axis=[1,2,3])
    errors_model=tf.reduce_mean(tf.abs(res_fine_proj-res_model),axis=[1,2,3])
    
    fig,ax=plt.subplots()
    ax.plot(errors_HLL,color="b",label="HLL")
    ax.plot(errors_model,color="r",label="model")

    # if (np.max(errors_model)>0.1):
    #     ax.set_ylim(0,0.1)
    
    ax.legend()
    ax.set_xlabel("time")

    plt.show()

### Shock waves

In [ ]:
def shock_waves(batch_size):
    k=k_tf32
    x = k.arange_float(param.xmin, param.xmax, param.dx)
    rho = np.array([k.where_float(x < 0.15, 0.5 + 0.5 * (n + 1), 0.5) for n in range(batch_size)])
    return rho[..., k.newaxis]

In [ ]:
w_init = shock_waves(5)
nb_t_shock = 2 * nb_t

param.BC_model = param.BC_solver = param.BC_neumann
W_val = core.compute_solutions(param, nb_t_shock, w_init, False, k_tf32)
compare_along_the_time(W_val, 0 * nb_t_shock // 2)

### Rarefaction waves

In [ ]:
def rarefaction_waves(batch_size):
    k=k_tf32
    x = k.arange_float(param.xmin, param.xmax, param.dx)
    rho = np.array([k.where_float(x < 0.15, 0.5, 0.5 + 0.5 * (n + 1)) for n in range(batch_size)])
    return rho[..., k.newaxis]

In [ ]:
w_init = rarefaction_waves(5)
nb_t_rarefaction = 2 * nb_t

param.BC_model = param.BC_solver = param.BC_neumann
W_val = core.compute_solutions(param, nb_t_rarefaction, w_init, False, k_tf32)
compare_along_the_time(W_val, 0 * nb_t_rarefaction // 2)

### Periodic

In [ ]:
w_init = init_periodic(param, 50)
W_val = core.compute_solutions(param, nb_t, w_init, False, k_tf32)
compare_along_the_time(W_val)

### Non-periodic

In [ ]:
param.BC_model = param.BC_solver = Param.BC_periodic  # todo=> reflexive
w_init = init_non_periodic(param, 50)
W_val = core.compute_solutions(param, nb_t, w_init, False, k_tf32)
compare_along_the_time(W_val)